In [ ]:
suppressWarnings(suppressPackageStartupMessages({
    library(Seurat)
    library(EnsDb.Hsapiens.v86)
    library(dplyr)
    library(ggplot2)
    library(bedr)
    library(SeuratDisk)
    library(scales)
    library(reshape2)
    library(Hmisc)
    library(tidyr)
    library(tidyverse)
    library(crayon)
    library(readr)
    library(future)
    library("TxDb.Hsapiens.UCSC.hg38.knownGene")
    library(GenomicRanges)
    library(GenomicFeatures)
    library(rsnps)
    library(biomaRt)
    library(ggpubr)
    library(gridExtra)
    library(patchwork)
    library(EnhancedVolcano)
    library(rlist)
    library(purrr)
    library(edgeR)
}))
options(timeout=100000)
source('/home/vsevim/prj/workflows/ckd/secondary/helper_functions_for_diffex.r')

# Definitions etc

In [ ]:
set.seed(1234)
options(digits=2)
stats <- c()

In [ ]:
options(future.globals.maxSize= 250 * 1024^3) # 650Gb
# plan()

# Enable parallelization
plan(sequential)
plan("multicore", workers = 32)

In [ ]:
if (!exists("papermill_run")) {
    prj_name <- "Screen1"
    secondary_a_path <- "S1/analysis/secondary/"
    save_seurat_h5 <- "YES"
    de_testing <- "NEIGHBORHOOD"
}


In [ ]:
custom_theme <- theme(
  plot.title = element_text(size=16, hjust = 0.5), 
  legend.key.size = unit(0.7, "cm"), 
  legend.text = element_text(size = 14))

### Load guide df

In [ ]:
df_guide <- read.table(
    "primary/S1_resources/66CRISPRi_ref_for_diffex.txt",
    sep = "\t", header = T, strip.white = T
)
integrated_h5_path <-
    "S1/analysis/secondary/integrated/seurat_objects/integrated.h5seurat"
neighbors_list <- list.load(
    "primary/S1_resources/neighbors_list.rds"
)

sample_n(df_guide, 6)

### Load Seurat file

In [ ]:
seurat_combined    <- LoadH5Seurat(integrated_h5_path, verbose = F)
seurat_combined$sct_cas9 = seurat_combined[['SCT']]@data['dCas9',]

Normalize RNA counts by <font color='red'>NormalizeData</font>  before running FindMarkers

In [ ]:
seurat_rna = CreateSeuratObject(seurat_combined[['RNA']])
seurat_rna <- NormalizeData(seurat_rna)
seurat_rna@meta.data <- seurat_combined@meta.data

### Load THRESHOLDS.tsv

In [ ]:
thr_f_name = paste0(secondary_a_path, "/integrated/THRESHOLDS.tsv")
df_thresholds = read.table(thr_f_name, header=T, strip.white = T)
df_thresholds <- df_thresholds %>% pivot_wider(names_from = batch, values_from = threshold)
df_thresholds <- as.data.frame(df_thresholds)

## Select perturbed/control cells
Find guide+ (perturbed) and guide- (unperturbed) cells for each guide

In [ ]:
libraries = unique(seurat_combined$library)
seurat_libs = list()
for(i in seq_along(libraries)){ 
    lib = libraries[i]
    seurat_libs[[i]] = subset(seurat_combined, subset = library == lib)
}
names(seurat_libs) <- libraries

In [ ]:
THRESHOLD_MULTIPLIER = 1
cat(red("Using ", THRESHOLD_MULTIPLIER, "x threshold"))

perturbed_cells_by_guide <- list()

for(i in 1:nrow(df_thresholds)){  
    perturbed_cells_in_all_libs = list()
    guide = df_thresholds$guide[i]
    # Loop over libraries
    for(lib in libraries){
        seurat_lib = seurat_libs[[lib]]
        threshold = THRESHOLD_MULTIPLIER * df_thresholds[i, lib]
        #cat(blue(guide, lib, threshold, "\n"))
        cells_in_lib = Cells(seurat_lib)        
        sgrna_counts = seurat_lib[['sgRNA']]@counts
        select_perturbed = sgrna_counts[guide, cells_in_lib] >= threshold
        perturbed_cells_in_library = cells_in_lib[select_perturbed]
        if(!is.na(threshold)) {
            perturbed_cells_in_all_libs =
                append(perturbed_cells_in_all_libs, perturbed_cells_in_library)
        }
    }
    perturbed_cells_by_guide[[i]] = perturbed_cells_in_all_libs
}
names(perturbed_cells_by_guide) <- df_thresholds$guide

In [ ]:
length(perturbed_cells_by_guide[[3]])

In [ ]:
df_thresholds$guide[grepl("NT" , df_thresholds$guide)]

# Check DE for genes near SNPs

In [ ]:
select_distal  = df_guide$subclass == 'ckd_de'
df_snps = unique(df_guide[select_distal, c('gene','alias')])

# Run diffex on pseudobulked samples using EdgeR

In [ ]:
df_targets = unique(filter(df_guide, subclass=='ckd_de')[,c('alias','gene','subclass', 'label')])
df_targets = unique(filter(df_guide, class=='targeting')[,c('alias','gene','subclass', 'label')])
#df_targets = unique(filter(df_guide, alias=='PLIN3'))
df_neighbor_de = NULL
df_cell_counts = NULL
test_use = "LR" #"MAST"  #"wilcox" "LR"
guides_to_skip = c()  #c("DE6", "DE15")  
logfc_threshold = 0.01
n_tests = 0

for(i in 1:nrow(df_targets)) {
    is_de = FALSE
    target          = df_targets[i,'alias']
    target_subclass = df_targets[i, 'subclass']
    label           = df_targets[i, 'label']
    snp_id_or_gene_name = df_targets[i,'gene']
    
    if(!str_detect(target, regex("^DE\\d+$"))) {
        is_de = FALSE
        neighbors = neighbors_list[[target]]
    } else {
        is_de = TRUE
        neighbors = neighbors_list[[snp_id_or_gene_name]]
    }

    if((target %in% guides_to_skip) | length(neighbors) == 0 ){
        cat(red("Skipping", target, snp_id_or_gene_name, length(neighbors), "\n"))
        next
    }
    guides_4_target = get_guides_by_subclass(df_guide, 'alias', target)
    cat(blue(target, target,":"), paste(guides_4_target, collapse=","),"\n")
    seurat_dummy <- mark_target_pos_neg(seurat_rna, perturbed_cells_by_guide, guides_4_target, print_counts = T)
    seurat_dummy$perturbation_status = Idents(seurat_dummy)
    print(table(filter(seurat_dummy@meta.data, perturbation_status == 'target_positive')$library) )

    # Create pseudobulk sample
    cluster_list = as.character(seurat_dummy$perturbation_status)
    cluster_list[cluster_list == "target_negative"] = 1
    cluster_list[cluster_list == "target_positive"] = 0
    seurat_dummy$cluster = as.factor(cluster_list)
    y = Seurat2PB(seurat_dummy, sample = "library", cluster = 'cluster')

    # Add target+/- counts to df_cell_counts
    if(!is_de) {
        df_dummy = as.data.frame(y$counts[target,])
        colnames(df_dummy) = "count"
        df_dummy$target <- target
        df_dummy$label = rownames(df_dummy)
        df_dummy$subclass = target_subclass
        df_samples = y$samples
        df_dummy = merge(df_dummy, df_samples, by = 0)
        df_cell_counts = rbind(df_cell_counts, df_dummy)
    }

    # Filter out small samples, lowly expressed genes.
    keep.samples <- y$samples$lib.size > 5e4
    table(keep.samples)
    y <- y[, keep.samples]
    keep.genes <- filterByExpr(
        y,
        group = y$samples$cluster,
        min.count = 3,
        min.total.count = 3
    )
    y <- y[keep.genes, ,keep = FALSE]
    table(keep.genes)
    y <- normLibSizes(y)

    # Create design matrix
    library <- factor(y$samples$sample)
    cluster <- as.factor(y$samples$cluster)
    design <- model.matrix(~ cluster + library)
    colnames(design) <- gsub("library", "", colnames(design))
    colnames(design)[1] <- "Int"

    ncls <- nlevels(cluster)
    contr <- rbind( matrix(1/(1-ncls), ncls, ncls), matrix(0, ncol(design)-ncls, ncls) )
    diag(contr) <- 1
    contr[1,] <- 0
    rownames(contr) <- colnames(design)
    colnames(contr) <- paste0("cluster", levels(cluster))
    
    # Estimate dispersion
    y <- estimateDisp(y, design, robust=TRUE)
    y$common.dispersion

    # Fit
    fit <- glmQLFit(y, design, robust=TRUE)
    qlf <- glmQLFTest(fit, contrast=contr[,1])
    options(repr.plot.width = 7, repr.plot.height = 5)
    plotQLDisp(fit)
    n_tests = n_tests + length(neighbors)

    if(de_testing == "NEIGHBORHOOD") {
        markers <- qlf$table[neighbors, ] %>% arrange(PValue)
    } else { 
        # Test all genes
        markers <- qlf$table %>% arrange(PValue)
    }
    
    markers$de_gene = rownames(markers)
    markers$target = target
    markers$subclass = target_subclass
    markers$label = label
    rownames(markers) <- NULL
    df_neighbor_de = rbind(df_neighbor_de, markers)
    print(head(markers))
    cat("-------------------------------------------------------\n\n")
}


### Correct p-values by total number of comparisons

In [ ]:
#n_tests = length(flatten(neighbors_list)) * length(df_targets)
print(n_tests)
df_neighbor_de$p_val_adj =  p.adjust(
    p = df_neighbor_de$PValue,
    method = "bonferroni",
    n = n_tests
)

In [ ]:
head(df_neighbor_de)

In [ ]:
df_cell_counts_backup <- df_cell_counts

In [ ]:
head(df_cell_counts)

In [ ]:
df_cell_counts = df_cell_counts_backup
df_cell_counts$label = str_replace(df_cell_counts$label, "Lib_._", "")
df_cell_counts$label = str_replace(df_cell_counts$label, "cluster0", "target(+)")
df_cell_counts$label = str_replace(df_cell_counts$label, "cluster1", "target(-)")

df_cell_counts$norm_count = max(df_cell_counts$lib.size) * df_cell_counts$count/df_cell_counts$lib.size

df_cell_counts_tss = df_cell_counts %>% filter(subclass == 'tss')
df_cell_counts_de  = df_cell_counts %>% filter(subclass == 'de_control')

options(repr.plot.width=16, repr.plot.height=6)
ggplot(df_cell_counts_tss, aes(x = label, y = norm_count, fill = sample)) +
    geom_bar(position = position_dodge2(padding = 0.3), width = 0.75, stat="identity") +
    facet_wrap(~ target, scales="free", ncol = 6) +
    theme(
        axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1, size = 12), 
        strip.text = element_text(size=10)
    ) 

options(repr.plot.width=18, repr.plot.height=6)
ggplot(df_cell_counts_de, aes(x = label, y = norm_count, fill = sample)) +
    geom_bar(position = position_dodge2(padding = 0.3), width = 0.75, stat="identity") +
    facet_wrap(~ target, scales="free", ncol = 7) +
    theme(
        axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1, size = 12), 
        strip.text = element_text(size=10)
    ) 

#geom_bar(position = position_dodge2(padding = 0.3), stat="identity")  +

In [ ]:
head(df_cell_counts)

## Inpect results

Print top high-confidence hits

In [ ]:
df_neighbor_de %>% filter(logFC < 0) %>% arrange(PValue) %>% head(25)

In [ ]:
df_neighbor_de %>% filter(logFC < 0) %>% filter(subclass == "tss") %>% filter(de_gene == target) %>% arrange(PValue) %>% head(25)

## Volcano Plot

In [ ]:
colors <- ifelse(
    df_neighbor_de$subclass == 'tss', 'black',
    ifelse(df_neighbor_de$subclass == 'de_control', 'gray', 'green') 
)
names(colors) <- df_neighbor_de$label

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 12)
EnhancedVolcano(df_neighbor_de, 
                lab = df_neighbor_de$de_gene,
                x ="logFC",
                y = "p_val_adj",
                title = paste(prj_name, "hits by target"),
                subtitle = 'Pseudobulk, EdgeR', 
                colCustom = colors,
                drawConnectors = TRUE,
                arrowheads = FALSE,
                pCutoff = 10e-04,
                FCcutoff = logfc_threshold,
                pointSize = 5.0,
                labSize = 4.0
                ) +
                xlim(-2, 2) +
                xlab(expression(paste("Average ", Log[2], " Fold Change"))) +
                ylab(expression(paste("\u2013", Log[10], " ", italic(p))))

# lab = df_neighbor_de$de_gene,
# drawConnectors = TRUE,

### Plot just the control TSS

In [ ]:
de_filter = (df_neighbor_de$subclass == 'tss') & (df_neighbor_de$target == df_neighbor_de$de_gene)
df_neighbor_de_subset = df_neighbor_de[de_filter ,]

options(repr.plot.width = 20, repr.plot.height = 12)
p1 = EnhancedVolcano(df_neighbor_de_subset, 
                lab = df_neighbor_de_subset$de_gene,
                x ="logFC",
                y = "p_val_adj",
                title = paste(prj_name, 'hits by target'),
                subtitle = 'only TSS controls, only hit==target, pseudobulk, EdgeR', 
                col = 'gray',
                drawConnectors = TRUE,
                arrowheads = FALSE,
                pCutoff = 10e-0,
                FCcutoff = logfc_threshold,
                pointSize = 5.0,
                labSize = 6.0
                ) +
                xlim(-2.5, 1) +
                xlab(expression(paste("Average ", Log[2], " Fold Change"))) +
                ylab(expression(paste("\u2013", Log[10], " ", italic(p))))


de_filter = (df_neighbor_de$subclass == 'tss')
df_neighbor_de_subset = df_neighbor_de[de_filter ,]
p2 = EnhancedVolcano(df_neighbor_de_subset, 
                lab = df_neighbor_de_subset$de_gene,
                x ="logFC",
                y = "p_val_adj",
                title = paste(prj_name, 'hits by target'),
                subtitle = 'only TSS controls, pseudobulk, EdgeR', 
                col = 'gray',
                drawConnectors = TRUE,
                arrowheads = FALSE,
                pCutoff = 10e-0,
                FCcutoff = logfc_threshold,
                pointSize = 5.0,
                labSize = 4.0
                ) +
                xlim(-2.5, 1) +
                xlab(expression(paste("Average ", Log[2], " Fold Change"))) +
                ylab(expression(paste("\u2013", Log[10], " ", italic(p))))

p1 + p2

In [ ]:
head(df_neighbor_de_subset)

### Plot just the control DE

In [ ]:
de_filter = (df_neighbor_de$subclass == 'de_control') & (df_neighbor_de$target == df_neighbor_de$de_gene)
df_neighbor_de_subset = df_neighbor_de[de_filter ,]

options(repr.plot.width = 20, repr.plot.height = 12)
p1 = EnhancedVolcano(df_neighbor_de_subset, 
                lab = df_neighbor_de_subset$de_gene,
                x ="logFC",
                y = "p_val_adj",
                title = paste(prj_name, 'hits by target'),
                subtitle = 'only DE controls, only hit==target, pseudobulk, EdgeR', 
                col = 'gray',
                drawConnectors = TRUE,
                arrowheads = FALSE,
                pCutoff = 10e-1,
                FCcutoff = logfc_threshold,
                pointSize = 5.0,
                labSize = 6.0
                ) +
                xlim(-1.5, 1) +
                xlab(expression(paste("Average ", Log[2], " Fold Change"))) +
                ylab(expression(paste("\u2013", Log[10], " ", italic(p))))


de_filter = (df_neighbor_de$subclass == 'de_control')
df_neighbor_de_subset = df_neighbor_de[de_filter ,]
p2 = EnhancedVolcano(df_neighbor_de_subset, 
                lab = df_neighbor_de_subset$de_gene,
                x ="logFC",
                y = "p_val_adj",
                title = paste(prj_name, 'hits by target'),
                subtitle = 'only DE controls, pseudobulk, EdgeR', 
                col = 'gray',
                drawConnectors = TRUE,
                arrowheads = FALSE,
                pCutoff = 10e-2,
                FCcutoff = logfc_threshold,
                pointSize = 5.0,
                labSize = 6.0
                ) +
                xlim(-1.5, 1) +
                xlab(expression(paste("Average ", Log[2], " Fold Change"))) +
                ylab(expression(paste("\u2013", Log[10], " ", italic(p))))

p1 + p2

### Plot just the CKD DE

In [ ]:
de_filter = (df_neighbor_de$label == 'CKD DE') 
df_neighbor_de_subset = df_neighbor_de[de_filter ,]

options(repr.plot.width = 10, repr.plot.height = 12)
p1 = EnhancedVolcano(df_neighbor_de_subset, 
                lab = df_neighbor_de_subset$de_gene,
                x ="logFC",
                y = "p_val_adj",
                title = paste(prj_name, 'hits by target'),
                subtitle = 'only CKD DE, pseudobulk, EdgeR', 
                col = 'gray',
                drawConnectors = TRUE,
                arrowheads = FALSE,
                pCutoff = 10e-2,
                FCcutoff = logfc_threshold,
                pointSize = 5.0,
                labSize = 6.0
                ) +
                xlim(-1.5, 0.4) +
                ylim(0, 25) +
                xlab(expression(paste("Average ", Log[2], " Fold Change"))) +
                ylab(expression(paste("\u2013", Log[10], " ", italic(p))))

p1 